# Computational Theory Assignment

In [ ]:
# Necessary imports

import numpy as np

## Problem 1

### SHA-1 Parity Function

The `Parity` function implements the SHA-1 Parity operation, which performs a bitwise XOR across three input values. This is commonly used in cryptographic hash functions to mix bits and increase unpredictability.

Each input should be an integer representing a binary value. The function returns the result as a binary string.

#### Example Usage

```python
result = Parity(0b1111, 0b1010, 0b0101)
print(result)  # Output: '0b0'
```

In this example, the function computes the XOR of the three binary numbers and returns the result in binary format.

#### Code

In [ ]:
def Parity(x, y, z):
    """
    SHA-1 Parity function.

    Parameters:
        x (int): first value (interpreted as binary).
        y (int): second value (interpreted as binary).
        z (int): third value (interpreted as binary).

    Returns:
        str: The result of the parity as a binary string.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # SHA-1 Parity function = bitwise XOR of each bit position
    result = x ^ y ^ z   # ^ is the bitwise XOR operator in Python

    return bin(result)


#### Testing

In [ ]:
def test_parity():
    assert Parity(0b0, 0b0, 0b0) == '0b0', "All zeros should return 0b0"
    assert Parity(0b1, 0b0, 0b0) == '0b1', "Single one should return 0b1"
    assert Parity(0b1, 0b1, 0b0) == '0b0', "Two ones should return 0b0"
    assert Parity(0b1, 0b1, 0b1) == '0b1', "Three ones should return 0b1"
    assert Parity(0b101, 0b010, 0b001) == '0b110', "Mixed bits test"
    assert Parity(0b1111, 0b1010, 0b0101) == '0b0', "4-bit test"
    print('All test cases passed.')

In [ ]:
# Run all tests
test_parity()

All test cases passed.


## Problem 2

## Problem 3

## Problem 4

## Problem 5

## End